# ArboLab Local Workflow Example

This notebook demonstrates how to use ArboLab locally in a pure Python environment. 
It covers:
1. Imports
2. Lab Initialization
3. Entity Management (CRUD)
4. Querying and Exploration

## 1. Imports and Setup

First, we set up logging and import the central `Lab`. We also perform cleanup to ensure a fresh run (preserving inputs).

In [ ]:
import shutil
from datetime import datetime
from pathlib import Path

from arbolab.core.security import LabRole
from arbolab.lab import Lab
from arbolab.models.core import Project, Tree
from sqlalchemy import select
from arbolab_logger import LoggerConfig, configure_logger, get_logger

# Clean up previous run (delete workspace/results, keep input)
base_path = Path("./example_workspace").resolve()
shutil.rmtree(base_path / "workspace", ignore_errors=True)
shutil.rmtree(base_path / "results", ignore_errors=True)

# Ensure input exists
(base_path / "input").mkdir(parents=True, exist_ok=True)

# Configure Logging (ArboLab will automatically add file logging upon initialization)
configure_logger(LoggerConfig(
    level="DEBUG",
    colorize=True, 
))

<Logger arbolab (DEBUG)>

## 2. Lab Initialization

We initialize a Lab by specifying a `base_root`. ArboLab will automatically create a clean structure:
- `workspace/`: For the database and config
- `input/`: For raw data ingestion
- `results/`: For analysis outputs

In [10]:
# Open the Lab using base_root to enforce standard structure
# Note: workspace_root must be explicitly None when using base_root due to API signature
lab = Lab.open(workspace_root=None, base_root=base_path, role=LabRole.ADMIN)

[2026-01-18 13:46:59] DEBUG    Lab Roots Configuration:                                                  ]8;id=378628;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=981812;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#192\192]8;;\

[2026-01-18 13:46:59] DEBUG      Workspace Root:                                                         ]8;id=181236;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=367426;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#193\193]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exampl           
                               e_workspace/workspace                                                               

[2026-01-18 13:46:59] DEBUG      Input Root:                                                             ]8;id=422474;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=559514;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#194\194]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exampl           
                               e_workspace/input                                                                   

[2026-01-18 13:46:59] DEBUG      Results Root:                                                           ]8;id=954832;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=572230;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#195\195]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exampl           
                               e_workspace/results                                                                 

[2026-01-18 13:46:59] DEBUG    Loading configuration from                                                ]8;id=906911;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=431160;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#217\217]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exampl           
                               e_workspace/workspace                                                               

...

[2026-01-18 13:46:59] DEBUG    Catalog up to date (2026-01-08.01).                            ]8;id=162203;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/core/catalog_manager.py\catalog_manager.py]8;;\:]8;id=256252;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/core/catalog_manager.py#52\52]8;;\

[2026-01-18 13:46:59] DEBUG    [Session 139705346589424] Committing transaction...                  ]8;id=891982;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=308733;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#108\108]8;;\

[2026-01-18 13:46:59] DEBUG    [Session 139705346589424] Committed transaction successfully         ]8;id=423616;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=282251;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#110\110]8;;\

[2026-01-18 13:46:59] DEBUG    [Session 139705346589424] Session closed                             ]8;id=7898;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=35164;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#118\118]8;;\

[2026-01-18 13:46:59] INFO     Lab initialized at                                                        ]8;id=559077;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=45160;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#118\118]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/exampl           
                               e_workspace/workspace (Role: LabRole.ADMIN)                                         

## 3. Creating and Managing Entities

ArboLab uses a "Recipe-First" approach. Every modification is recorded, ensuring reproducibility. 
You can use `define_*`, `modify_*`, and `remove_*` methods directly on the `lab` object.

In [11]:
# 3.1 Create a Project
project = lab.define_project(
    name="Forest Monitoring 2026",
    description="Analyzing tree growth and health in the local test area."
)
print(f"Created Project: {project.name} (ID: {project.id})")

# 3.2 Create an Experiment within the project
experiment = lab.define_experiment(
    project_id=project.id,
    name="Spring Growth Campaign",
    start_time=datetime.now(),
    description="Initial measurements for the spring season."
)
print(f"Created Experiment: {experiment.name} (ID: {experiment.id})")

# 3.3 Create a 'Thing' (e.g., a Tree)
# First define the base 'Thing'
thing = lab.define_thing(
    project_id=project.id,
    name="Oak-01",
    kind="tree"
)

# Then add specific Tree metadata
tree = lab.define_tree(
    id=thing.id, # Must match the Thing ID
    height=1500, # in cm
    circumference=120
)
print(f"Created Tree with Thing ID: {thing.id}")

[2026-01-18 13:47:08] DEBUG    [Session 139706483974336] Started transaction                        ]8;id=612773;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=210322;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#105\105]8;;\

[2026-01-18 13:47:08] DEBUG    [Session 139706483974336] Committing transaction...                  ]8;id=210560;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=211280;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#108\108]8;;\

[2026-01-18 13:47:08] DEBUG    [Session 139706483974336] Committed transaction successfully         ]8;id=414277;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=745424;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#110\110]8;;\

[2026-01-18 13:47:08] DEBUG    [Session 139706483974336] Session closed                             ]8;id=929316;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=482688;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#118\118]8;;\

Created Project: Forest Monitoring 2026 (ID: 1)


...

[2026-01-18 13:47:08] DEBUG    [Session 139705346443552] Started transaction                        ]8;id=494773;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=42589;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#105\105]8;;\

[2026-01-18 13:47:08] ERROR    [Session 139705346443552] Transaction failed:                        ]8;id=618912;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=965991;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#112\112]8;;\
                               (duckdb.duckdb.ConstraintException) Constraint Error: Duplicate key                 
                               "id: 1" violates primary key constraint. If this is an unexpected                   
                               constraint violation please double check with the known index                       
                               limitations section in our documentation                                            
                               (https://duckdb.org/docs/sql/indexes).                                              
                               [SQL: INSERT INTO trees (id, species_id, height, circumference,                     
                               fork_height) VALUES ($1, $2, $3, $4, $5) RETURNING trees.created_at,                
                               trees.updated_at]                                                                   
                                                                                                                   
                               (Background on this error at: https://sqlalche.me/e/20/gkpj)                        

[2026-01-18 13:47:08] DEBUG    [Session 139705346443552] Rolled back transaction                    ]8;id=497614;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=423324;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#114\114]8;;\

[2026-01-18 13:47:08] DEBUG    [Session 139705346443552] Session closed                             ]8;id=286229;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=525096;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#118\118]8;;\

IntegrityError: (duckdb.duckdb.ConstraintException) Constraint Error: Duplicate key "id: 1" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).
[SQL: INSERT INTO trees (id, species_id, height, circumference, fork_height) VALUES ($1, $2, $3, $4, $5) RETURNING trees.created_at, trees.updated_at]
[parameters: (1, None, 1500, 120, None)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

### Modifying Entities
Updating data is just as easy. Changes are also recorded in the workspace recipe.

In [4]:
# Update the project description
lab.modify_project(
    id=project.id, 
    description="Updated: Now including more sensors."
)

# Update tree height
lab.modify_tree(
    id=tree.id,
    height=1550
)
print("Entities updated successfully.")

[2026-01-18 13:45:29] DEBUG    [Session 139706484150576] Started transaction                        ]8;id=413199;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=846902;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#105\105]8;;\

[2026-01-18 13:45:29] DEBUG    Updated Project: Forest Monitoring 2026                               ]8;id=127029;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=24503;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#89\89]8;;\

[2026-01-18 13:45:29] DEBUG    [Session 139706484150576] Committing transaction...                  ]8;id=429212;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=670653;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#108\108]8;;\

[2026-01-18 13:45:29] DEBUG    [Session 139706484150576] Committed transaction successfully         ]8;id=297644;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=588616;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#110\110]8;;\

[2026-01-18 13:45:29] DEBUG    [Session 139706484150576] Session closed                             ]8;id=47671;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=585570;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#118\118]8;;\

...

[2026-01-18 13:45:29] DEBUG    Updated Tree: Tree #1                                                 ]8;id=8904;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=598181;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#89\89]8;;\

[2026-01-18 13:45:29] DEBUG    [Session 139705346450368] Committing transaction...                  ]8;id=296971;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=332753;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#108\108]8;;\

[2026-01-18 13:45:29] DEBUG    [Session 139705346450368] Committed transaction successfully         ]8;id=418220;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=883993;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#110\110]8;;\

[2026-01-18 13:45:29] DEBUG    [Session 139705346450368] Session closed                             ]8;id=930498;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=951005;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#118\118]8;;\

Entities updated successfully.


## 4. Querying Data

You can access the internal database session for advanced querying using SQLAlchemy.

In [5]:
with lab.database.session() as session:
    # Query all projects
    stmt = select(Project)
    projects = session.execute(stmt).scalars().all()
    
    print(f"Found {len(projects)} projects in the Lab:")
    for p in projects:
        print(f" - {p.name} (ID: {p.id})")

    # Query trees with height > 1000
    stmt = select(Tree).where(Tree.height > 1000)
    big_trees = session.execute(stmt).scalars().all()
    print(f"\nTrees taller than 10m: {len(big_trees)}")

[2026-01-18 13:45:35] DEBUG    [Session 139705346444560] Started transaction                        ]8;id=862436;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=378838;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#105\105]8;;\

Found 1 projects in the Lab:
 - Forest Monitoring 2026 (ID: 1)

Trees taller than 10m: 1


[2026-01-18 13:45:35] DEBUG    [Session 139705346444560] Committing transaction...                  ]8;id=652476;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=495806;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#108\108]8;;\

[2026-01-18 13:45:35] DEBUG    [Session 139705346444560] Committed transaction successfully         ]8;id=296669;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=670674;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#110\110]8;;\

[2026-01-18 13:45:35] DEBUG    [Session 139705346444560] Session closed                             ]8;id=6687;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=575511;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#118\118]8;;\

## 5. What else can ArboLab do?

- **Recipes**: All your actions are stored in `current.json`. You can share this recipe to reproduce your exact workspace state elsewhere.
- **Plugins**: Discover and use device plugins (like linescale or treeqinetic).
- **Data Storage**: Local Parquet/DuckDB storage for high-performance time-series data.

In [6]:
lab.close()

[2026-01-18 13:46:31] DEBUG    Database engine disposed.                                            ]8;id=460300;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py\database.py]8;;\:]8;id=96710;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/database.py#138\138]8;;\

[2026-01-18 13:46:31] DEBUG    Lab connected to                                                           ]8;id=769138;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py\lab.py]8;;\:]8;id=627559;file:///mnt/data/dev/packages/arbolab_mvp/packages/arbolab/src/arbolab/lab.py#92\92]8;;\
                               /mnt/data/dev/packages/arbolab_mvp/examples/arbolab/local_workflow/example          
                               _workspace/workspace closed.                                                        